# Capstone: Exploratory Prediction Modeling

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns


# Import utilities
# import pathlib
import time

# Export dataFrame's as images
import dataframe_image as dfi

# import project utils
import sys
sys.path.append('../src')

import data_utils
from data_utils import Config

import graph_utils

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, roc_auc_score, log_loss
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_curve

from xgboost import XGBClassifier
import xgboost as xgb

In [3]:
# Configure logging
import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# logging.getLogger().setLevel(logging.DEBUG)
# logging.getLogger().setLevel(logging.INFO)

In [4]:
def time_secs_to_msg(lapse_time_secs, mins_label='m', secs_label='s'):
    if lapse_time_secs <= 60:
        return f'{lapse_time_secs%60:.2f}{secs_label}'
    else:
        return f'{lapse_time_secs//60:,.0f}{mins_label} {lapse_time_secs%60:.2f}{secs_label}'

## The Data: San Francisco Police Department Incident Reports

### Read the Data

In [7]:
# Which dataset to work from?

sample_file = data_utils.select_sample_csv_file(pct=100)
# sample_file = data_utils.select_sample_csv_file(pct=10)
print(f'Selected sample file: {sample_file}')

Selected sample file: ../data/incidents_clean.csv


In [8]:
current_raw_df, current_clean_df = data_utils.get_clean_data_from_csv(sample_file)

Reading file: ../data/incidents_clean.csv ... Done: 894,585 rows, 36 columns
... Converting datetime to timeseries ... Done
... Setting index to datetime ... Done
Done


In [9]:
data = data_utils.preprocess_data(current_raw_df.copy())

Pre-processing ... 
... Dropping unwanted columns ... 
... preprocess_drop_cols: Column Unnamed: 0 not dropped: KeyError("['Unnamed: 0'] not found in axis")
... preprocess_drop_cols: Column esncag_-_boundary_file dropped
... preprocess_drop_cols: Column central_market/tenderloin_boundary_polygon_-_updated dropped
... preprocess_drop_cols: Column civic_center_harm_reduction_project_boundary dropped
... preprocess_drop_cols: Column hsoc_zones_as_of_2018-06-05 dropped
... preprocess_drop_cols: Column invest_in_neighborhoods_(iin)_areas dropped
... preprocess_drop_cols: Column report_type_code dropped
... preprocess_drop_cols: Column report_type_description dropped
... preprocess_drop_cols: Column filed_online dropped
... preprocess_drop_cols: Column intersection dropped
... preprocess_drop_cols: Column cnn dropped
... preprocess_drop_cols: Column point dropped
... preprocess_drop_cols: Column supervisor_district dropped
... preprocess_drop_cols: Column supervisor_district_2012 dropped
...

In [10]:
# Fix data value artifacts that were discovered during EDA
data = data_utils.fix_data_artifacts(data)

Fixing data artifacts (in-place) ... 
... Category column:
    ..."Human Trafficking*"
    ..."Motor Vehicle Theft"
    ..."Weapons Offence"
Done


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 829328 entries, 2023-03-11 14:00:00 to 2023-03-21 17:42:00
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   date             829328 non-null  object 
 1   time             829328 non-null  object 
 2   year             829328 non-null  int64  
 3   day_of_week      829328 non-null  object 
 4   category         829328 non-null  object 
 5   resolution       829328 non-null  object 
 6   police_district  829328 non-null  object 
 7   neighborhood     829328 non-null  object 
 8   latitude         829328 non-null  float64
 9   longitude        829328 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 69.6+ MB


## Summary of EDA

After cleaning the data and performing basic EDA, we have established the following:

1. Target variable `category`
   * Evenly spread across time
   * Incidence of crimes is extremely skewed/unbalanced by category. Larceny (29.02%) by far outweighing the other top-10 categories with each being in the single digits
3. Features impacting `category`
   * Affected by incident time and date components: date, time, day of week, month, year, etc
   * Affected by police disctrict
   * Affect by latitude and logitude (TODO: need visualization)
4. We artificially removed nulls (TODO: will come back to impute data later)

## Feature Engineering

In [15]:
data.head(2)

,date,time,year,day_of_week,category,resolution,police_district,neighborhood,latitude,longitude
datetime,,,,,,,,,,
2023-03-11 14:00:00,2023/03/11,14:00,2023,Saturday,Assault,Open or Active,Park,Golden Gate Park,37.772895,-122.454285
2022-06-27 12:00:00,2022/06/27,12:00,2022,Monday,Lost Property,Open or Active,Central,Financial District/South Beach,37.787359,-122.408227


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 829328 entries, 2023-03-11 14:00:00 to 2023-03-21 17:42:00
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   date             829328 non-null  object 
 1   time             829328 non-null  object 
 2   year             829328 non-null  int64  
 3   day_of_week      829328 non-null  object 
 4   category         829328 non-null  object 
 5   resolution       829328 non-null  object 
 6   police_district  829328 non-null  object 
 7   neighborhood     829328 non-null  object 
 8   latitude         829328 non-null  float64
 9   longitude        829328 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 69.6+ MB


### Encoding: Time-based columns

Let's unpack the date and time into their components that are still missing so there is less to encode:

In [19]:
data['hour'] = data.index.map(lambda x: x.hour)
data['minute'] = data.index.map(lambda x: x.minute)
data['day'] = data.index.map(lambda x: x.day)
data['month'] = data.index.map(lambda x: x.month)

Now let's encode day_of_week to numeric values:

In [21]:
enc_dow = LabelEncoder()
enc_dow.fit(data.day_of_week.unique())
data['dow'] = enc_dow.transform(data.day_of_week)

Let's mark the redundant columns to be dropped after feature engineering:

In [23]:
drop_encoded_cols = ['date', 'time', 'day_of_week']

### Encoding: Resolution

We will also drop the resolution column since it doesn't impact crime prediction:

In [26]:
data.resolution.value_counts()

resolution
Open or Active          662581
Cite or Arrest Adult    166747
Name: count, dtype: int64

In [27]:
drop_encoded_cols.append('resolution')

### Encoding: Category

In [29]:
enc_cat = LabelEncoder()
enc_cat.fit(data.category.unique())
data.category = enc_cat.transform(data.category)

### Encoding: Police District

In [31]:
enc_pd = LabelEncoder()
enc_pd.fit(data.police_district.unique())
data['pd'] = enc_pd.transform(data.police_district)

### Encoding: Neighborhood

In [33]:
enc_hood = LabelEncoder()
enc_hood.fit(data.neighborhood.unique())
data.neighborhood = enc_hood.transform(data.neighborhood)

### Dropping Redundant Columns

We can now drop the redundant encoded columns:

In [36]:
drop_encoded_cols.append('police_district')

print(f'Dropping encoded columns: {drop_encoded_cols}')
data.drop(columns=drop_encoded_cols, inplace=True)

Dropping encoded columns: ['date', 'time', 'day_of_week', 'resolution', 'police_district']


In [37]:
data.head(2)

,year,category,neighborhood,latitude,longitude,hour,minute,day,month,dow,pd
datetime,,,,,,,,,,,
2023-03-11 14:00:00,2023,1,7,37.772895,-122.454285,14,0,11,3,2,5
2022-06-27 12:00:00,2022,18,5,37.787359,-122.408227,12,0,27,6,1,1


In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 829328 entries, 2023-03-11 14:00:00 to 2023-03-21 17:42:00
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          829328 non-null  int64  
 1   category      829328 non-null  int64  
 2   neighborhood  829328 non-null  int64  
 3   latitude      829328 non-null  float64
 4   longitude     829328 non-null  float64
 5   hour          829328 non-null  int64  
 6   minute        829328 non-null  int64  
 7   day           829328 non-null  int64  
 8   month         829328 non-null  int64  
 9   dow           829328 non-null  int64  
 10  pd            829328 non-null  int64  
dtypes: float64(2), int64(9)
memory usage: 75.9 MB


## Data Preparation

### Create Train/Test Splits

In [41]:
X = data.drop('category', axis='columns')
y = data['category']

In [42]:
# OneHot Encode the features and drop the first value to reduce multicollinearity
X = pd.get_dummies(X, drop_first=True)

In [43]:
# Consistent random_state for the project
print(f'Project-wide random_state: {Config.RANDOM_STATE}')

Project-wide random_state: 42


In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    stratify=y, random_state=Config.RANDOM_STATE)

In [45]:
print('AFTER TRAIN_TEST_SPLIT: Data{}, X_train{}, X_test{}, y_train{}, y_test{}'
      .format(data.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape))

AFTER TRAIN_TEST_SPLIT: Data(829328, 11), X_train(663462, 10), X_test(165866, 10), y_train(663462,), y_test(165866,)


In [46]:
# spot-check feature encoding
X.T.iloc[:, 0:5]

datetime,2023-03-11 14:00:00,2022-06-27 12:00:00,2023-03-16 17:30:00,2023-03-21 15:50:00,2021-08-22 09:40:00
year,2023.000000,2022.000000,2023.000000,2023.000000,2021.000000
neighborhood,7.000000,5.000000,28.000000,35.000000,26.000000
latitude,37.772895,37.787359,37.762290,37.787038,37.793977
longitude,-122.454285,-122.408227,-122.401324,-122.418271,-122.429804
hour,14.000000,12.000000,17.000000,15.000000,9.000000
minute,0.000000,0.000000,30.000000,50.000000,40.000000
day,11.000000,27.000000,16.000000,21.000000,22.000000
month,3.000000,6.000000,3.000000,3.000000,8.000000
dow,2.000000,1.000000,4.000000,5.000000,3.000000
pd,5.000000,1.000000,0.000000,4.000000,4.000000


### Feature Scaling

In [48]:
# Scale the data - we'll use StandardScaler for the baseline model
logging.debug('Scaling data')
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('AFTER SCALING: Data{}, X_train_scaled{}, X_test_scaled{}, y_train{}, y_test{}'
      .format(data.shape, X_train_scaled.shape, X_test_scaled.shape, y_train.shape, y_test.shape))

AFTER SCALING: Data(829328, 11), X_train_scaled(663462, 10), X_test_scaled(165866, 10), y_train(663462,), y_test(165866,)


## Model Exploration

The task of classifying the incident types based on a set of historical attrirbutes (features) and predicting on similar attributes is a **multiclass classification** problem. We will now experiment on some ML models that are generally used for similar problems to see what would be the best choice for us.

We will evaluate the following models:

* Simple classification models
  * `DummyClassifier` to get a baseline for our project
  * `LogisticRegression` with L1 Regularization
* Multiclass classifiers
  * `KNeighborsClassifier`
* Ensemble methods: Since our dataset has high variability with a lot of numerical and cagtegorical features with a range of mean and variance, we plan to employ ensemble methods and tune them for best results
  * `RandomForestClassifier`
  * `XGBClassifier`: We considered `XGLite` but selected XGBoost as it provides better model explainability features like SHAP values, which we expect to be able to use in explaining our results

We will now evaluate different models for predicting the Crime Category from our features:

In [52]:
X_train.columns

Index(['year', 'neighborhood', 'latitude', 'longitude', 'hour', 'minute',
       'day', 'month', 'dow', 'pd'],
      dtype='object')

### Evaluation Metrics

In this project, we are predicting or classifyig across 49 crime categories. We will use two evaluation metrics to compare our models:

1. **Accuracy**: Measures the proportion of correct predictions over all predictions made. The accuracy benchmark is 1/49 or 2.04% given our crime categories
2. **Log_Loss**: Measures the accuracy of a classifier by penalizing false classifications. It does this by taking the negative logarithm of the predicted probability for the true class. The goal is to minimize this loss, meaning that higher probabilities are assigned to the correct classes. Log loss is a powerful way to evaluate not just if the model is making the right predictions, but how confident it is in those predictions. A lower log loss indicates a model that is both accurate and confident.
   * TODO: Benchmark???

While accuracy provides a simple measure of correctness, log-loss offers a more nuanced view by considering how confident those predictions are. A model that predicts with 51% confidence for the correct class will have the same accuracy as one that predicts with 99% confidence, but their log loss will be very different. The 99%-confident model will have a much lower log loss.

We'll use them together for a comprehensive evaluation and to learn more about them.

The `build_results_row` utility function will be used to standardize the recording and reporting of our model exploration: 

In [56]:
def build_results_row(name, model, Xtr, Xte, ytr, yte, use_best=False):
    """
    Given the model and training/test sets, builds a row of metrics for reporting the results

    :param name: Name/Description of model
    :param model: Fully constructed model instance - will call fit() and predict() to get metrics
    :param Xtr: X_train - scale before calling
    :param Xte: X_test - scale before calling
    :param ytr: Y_train set
    :param yteL: Y_test set
    """
    
    print(f'{name}: Starting', flush=True)
    start_time = time.time()

    # train the model
    clf = model.fit(Xtr, ytr)

    # if we're tuning then use best_estimator
    if use_best:
        clf = model.best_estimator_
        logging.debug(f'{name}: Best Model={clf}')
        logging.debug(f'{name}: Best Params={model.best_params_}')

    # Save fit time
    fit_time = time.time() - start_time
    logging.debug(f'{name}: Fitted')

    # get the predictions / probabilities
    y_preds = clf.predict(Xte)
    y_probs_full = clf.predict_proba(Xte)
    y_probs = y_probs_full[:, 1]

    # logging.debug(f'>>> yte.shape{yte.shape} y_preds.shape{y_preds.shape} y_probs_full.shape{y_probs_full.shape}')
    # logging.debug(f'>>> {np.unique(yte)}')
    # logging.debug(f'>>> {np.unique(y_preds)}')
    # logging.debug(f'>>> {np.unique(y_probs_full)}')
    logging.debug(f'{name}: Got preds/probs')

    cm = confusion_matrix(yte, y_preds)
    logging.debug(f'{name}: cm.shape: {cm.shape}')

    # Get metrics
    row = {
        'Train Time': time_secs_to_msg(fit_time),
        'Train Accuracy': f'{model.score(Xtr, ytr)*100:.2f}%',
        'Test Accuracy': f'{model.score(Xte, yte)*100:.2f}%',
        'Precision': f'{precision_score(yte, y_preds, average="weighted")*100:.2f}%',  # for multi-class with imbalance
        'Recall': f'{recall_score(yte, y_preds, average="weighted")*100:.2f}%',
        'F1': f'{f1_score(yte, y_preds, average="weighted")*100:.2f}%',
        'AUC': f'{roc_auc_score(yte, y_probs_full, average="weighted", multi_class="ovr")*100:.2f}%',    # faster with imbalanced multi-class cases
        'LogLoss': f'{log_loss(yte, y_probs_full, labels=np.unique(yte)):.4f}',
        'preds': y_preds,
        'probs': y_probs,
        'cm': cm,
        # 'TN': f'{cm[0, 0]:,d}',
        # 'FP': f'{cm[0, 1]:,d}',
        # 'FN': f'{cm[1, 0]:,d}',
        # 'TP': f'{cm[1, 1]:,d}',
        'params': model.get_params(),
        'best_params': None,
        'best_model': clf,
    }
    if use_best:
        row.update({'best_params': model.best_params_})
        
    logging.debug(f'{name}: Got metrics')
    
    print(f'{name}: Done: {time_secs_to_msg(time.time()-start_time)}')

    return row

### Establishing a Baseline

#### DummyClassifier

We will use the Scikit-Learn DummyClassifier method to get a baseline for our predictions using the different strategies provided by the library:

In [60]:
n_estimators = 100

In [61]:
# let's start saving the results for reporting out
results_defaults = {}

# Reports won't print all the columns
report_cols = ['Train Time', 
               'Train Accuracy', 'Test Accuracy', 'LogLoss',
               'Precision', 'Recall', 'F1', 'AUC']

In [62]:
# Define the DummyClassifier models we want to test
models_dummy = {
    'DummyClassifier: uniform': DummyClassifier(strategy='uniform', random_state=Config.RANDOM_STATE),
    'DummyClassifier: most_frequent': DummyClassifier(strategy='most_frequent', random_state=Config.RANDOM_STATE),
    'DummyClassifier: stratified': DummyClassifier(strategy='stratified', random_state=Config.RANDOM_STATE),
}

In [63]:
# logging.getLogger().setLevel(logging.DEBUG)

# Get metrics row for the report - will fit() and predict() to generate metrics
for name, model in models_dummy.items():
    results_defaults[name] = build_results_row(name, model, X_train_scaled, X_test_scaled, y_train, y_test)

# logging.getLogger().setLevel(logging.INFO)

DummyClassifier: uniform: Starting
DummyClassifier: uniform: Done: 3.96s
DummyClassifier: most_frequent: Starting


/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


DummyClassifier: most_frequent: Done: 2.19s
DummyClassifier: stratified: Starting
DummyClassifier: stratified: Done: 16.75s


The warning above is from the precision calculation within scikit-learn, and highlights that some labels have no predicted samples, which results in precision being undefined for those labels. We can ignore the warning since we're using accuracy as our key evaluation metric. We could use `prescion_score(zero_division=0)` to suppress the warning, but we'll ignore it instead to ensure we're aware of the condition.

In [65]:
# View results by highest Test Accuracy and lowest log_loss
results_defaults_df = pd.DataFrame(results_defaults).T.sort_values(by=['Test Accuracy', 'LogLoss'], ascending=[False, True])

In [66]:
# format the output to hilite results
hilite=data_utils.Config.TBL_HILITE_COLOR
results_defaults_styled = results_defaults_df[report_cols].style.map(lambda val: f'background-color: {hilite}',
                                                                     subset=['Test Accuracy','LogLoss'])
results_defaults_styled = results_defaults_styled.set_table_styles({
    'LogLoss': [{'selector': 'th', 'props': [('background-color', hilite)]}],
    'Test Accuracy': [{'selector': 'th', 'props': [('background-color', hilite)]}]
}, overwrite=False)
results_defaults_styled

,Train Time,Train Accuracy,Test Accuracy,LogLoss,Precision,Recall,F1,AUC
DummyClassifier: most_frequent,0.09s,28.93%,28.93%,25.6175,8.37%,28.93%,12.98%,50.00%
DummyClassifier: uniform,0.10s,2.21%,2.21%,3.8067,11.62%,2.21%,3.12%,50.00%
DummyClassifier: stratified,0.09s,11.55%,11.44%,31.9203,11.46%,11.44%,11.45%,49.95%


#### Default Models

We will now explore the selected models with out-of-the-box default settings of their hyperparameters to get a baseline per model class.

In [69]:
# Define the default models
models_default = {
    'LogisticRegression (Default)': LogisticRegression(random_state=Config.RANDOM_STATE),
    'KNeighborsClassifier (Default)': KNeighborsClassifier(),
    'RandomForestClassifier (Default)': RandomForestClassifier(random_state=Config.RANDOM_STATE),
    'XGBClassifier (Default)': XGBClassifier(random_state=Config.RANDOM_STATE)
}

In [70]:
# logging.getLogger().setLevel(logging.DEBUG)

# Get metrics row for the report - will fit() and predict() to generate metrics
for name, model in models_default.items():
    results_defaults[name] = build_results_row(name, model, X_train_scaled, X_test_scaled, y_train, y_test)

# logging.getLogger().setLevel(logging.INFO)

LogisticRegression (Default): Starting


/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


LogisticRegression (Default): Done: 2m 30.01s
KNeighborsClassifier (Default): Starting


/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


KNeighborsClassifier (Default): Done: 10m 11.71s
RandomForestClassifier (Default): Starting
RandomForestClassifier (Default): Done: 10m 2.51s
XGBClassifier (Default): Starting


/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


XGBClassifier (Default): Done: 4m 54.55s


In [71]:
# View results by highest Test Accuracy and lowest log_loss
results_defaults_df = pd.DataFrame(results_defaults).T.sort_values(by=['Test Accuracy', 'LogLoss'], ascending=[False, True])

In [72]:
# format the output to hilite results
hilite=data_utils.Config.TBL_HILITE_COLOR
results_defaults_styled = results_defaults_df[report_cols].style.map(lambda val: f'background-color: {hilite}',
                                                                     subset=['Test Accuracy','LogLoss'])
results_defaults_styled = results_defaults_styled.set_table_styles({
    'LogLoss': [{'selector': 'th', 'props': [('background-color', hilite)]}],
    'Test Accuracy': [{'selector': 'th', 'props': [('background-color', hilite)]}]
}, overwrite=False)
results_defaults_styled

,Train Time,Train Accuracy,Test Accuracy,LogLoss,Precision,Recall,F1,AUC
RandomForestClassifier (Default),5m 35.47s,87.94%,33.97%,5.6816,28.66%,33.97%,29.36%,72.13%
XGBClassifier (Default),3m 58.44s,35.92%,33.64%,2.3761,27.36%,33.64%,24.63%,74.45%
LogisticRegression (Default),2m 25.31s,29.19%,29.19%,2.6363,13.94%,29.19%,14.65%,63.22%
DummyClassifier: most_frequent,0.09s,28.93%,28.93%,25.6175,8.37%,28.93%,12.98%,50.00%
KNeighborsClassifier (Default),1.93s,44.13%,24.83%,18.7904,19.91%,24.83%,20.96%,61.19%
DummyClassifier: uniform,0.10s,2.21%,2.21%,3.8067,11.62%,2.21%,3.12%,50.00%
DummyClassifier: stratified,0.09s,11.55%,11.44%,31.9203,11.46%,11.44%,11.45%,49.95%


In [73]:
# Save results to file for final report
if not Config.SUPPRESS_OUTPUT_FILES:
    dfi.export(results_defaults_styled, data_utils.Config.IMAGE_DIR / 'table_models_defaults.png')

### Model Optimization

In [75]:
# let's start saving the results for reporting out
results_tuned = {}

# Reports won't print all the columns
report_cols_tuned = ['params', 'Train Time', 
                     'Train Accuracy', 'Test Accuracy', 'LogLoss',
                     'Precision', 'Recall', 'F1', 'AUC']

In [76]:
# Define the default models
models_tuned = {
    'LogisticRegression': LogisticRegression(penalty='l1', solver='saga', max_iter=1000, 
                                             verbose=1, n_jobs=3, random_state=Config.RANDOM_STATE),
    'RandomForestClassifier': RandomForestClassifier(n_estimators=n_estimators, max_depth=15,
                                                     min_samples_leaf=5, min_samples_split=25, 
                                                     random_state=Config.RANDOM_STATE, 
                                                     verbose=1, n_jobs=2),
    'XGBClassifier': XGBClassifier(n_estimators=n_estimators, objective="multi:softprob", 
                                   n_jobs=2, random_state=Config.RANDOM_STATE),
}

In [77]:
# logging.getLogger().setLevel(logging.DEBUG)

# Get metrics row for the report - will fit() and predict() to generate metrics
for name, model in models_tuned.items():
    results_tuned[name] = build_results_row(name, model, X_train_scaled, X_test_scaled, y_train, y_test)

# logging.getLogger().setLevel(logging.INFO)

LogisticRegression: Starting


[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


convergence after 30 epochs took 124 seconds


/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


LogisticRegression: Done: 2m 27.10s
RandomForestClassifier: Starting


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  1.6min
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:  4.7min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    3.1s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    6.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    2.8s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    7.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   19.7s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:   39.5s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    2.7s
[Parallel(n_job

RandomForestClassifier: Done: 5m 50.11s
XGBClassifier: Starting


/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


XGBClassifier: Done: 14m 49.13s


In [78]:
# View results by highest Test Accuracy and lowest log_loss
results_tuned_df = pd.DataFrame(results_tuned).T.sort_values(by=['Test Accuracy', 'LogLoss'], ascending=[False, True])

In [79]:
# format the output to hilite results
hilite=data_utils.Config.TBL_HILITE_COLOR
results_tuned_styled = results_tuned_df[report_cols_tuned].style.map(lambda val: f'background-color: {hilite}', 
                                                                     subset=['Test Accuracy','LogLoss'])
results_tuned_styled = results_tuned_styled.set_table_styles({
    'params': [{'selector': 'td', 'props': [('max-width', '300px'), 
                                  ('white-space', 'normal'), 
                                  ('word-wrap', 'break-word')]}],
    'LogLoss': [{'selector': 'th', 'props': [('background-color', hilite)]}],
    'Test Accuracy': [{'selector': 'th', 'props': [('background-color', hilite)]}]
}, overwrite=False)
results_tuned_styled

,params,Train Time,Train Accuracy,Test Accuracy,LogLoss,Precision,Recall,F1,AUC
XGBClassifier,"{'objective': 'multi:softprob', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': None, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': None, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 100, 'n_jobs': 2, 'num_parallel_tree': None, 'random_state': 42, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}",10m 36.97s,35.92%,33.64%,2.3761,27.36%,33.64%,24.63%,74.45%
RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 15, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 5, 'min_samples_split': 25, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': 2, 'oob_score': False, 'random_state': 42, 'verbose': 1, 'warm_start': False}",4m 44.66s,35.67%,33.05%,2.4068,28.66%,33.05%,21.77%,74.16%
LogisticRegression,"{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 1000, 'multi_class': 'deprecated', 'n_jobs': 3, 'penalty': 'l1', 'random_state': 42, 'solver': 'saga', 'tol': 0.0001, 'verbose': 1, 'warm_start': False}",2m 3.84s,29.19%,29.20%,2.6362,14.15%,29.20%,14.66%,63.23%


In [80]:
# Save results to file for final report
if not Config.SUPPRESS_OUTPUT_FILES:
    dfi.export(results_tuned_styled, data_utils.Config.IMAGE_DIR / 'table_models_tuned.png')

### GridSearchCV